This program implements a Trivia Game where players take turns answering questions fetched from the OpenTrivia Database API. It uses the basics of object-oriented programming paired with a simple graphical user interface (GUI), built with tkinter.

Game Logic:

The game starts by prompting the user to input the number of players. For each player, the user provides a name and specifies whether the player is a child or an adult. It is important to note that this happens before the GUI is launched. This determines the difficulty of the questions for that player (children get easier questions).

The game proceeds in rounds, where each player takes a turn to answer a trivia question.
Answer options are displayed on the right side of the screen, and players select their answers.

A player earns 1 point for each correct answer.
The scores for all players are dynamically displayed in a leaderboard on the left side of the screen.

The game ends when the user clicks the "End Game" button, regardless of the number of rounds played. The game can thus, in theory, go on indefinetely.

First, the necessary libraries are imported. <u>**If the user does not have these installed already, they should download them using pip install [library]**

In [ ]:
import requests as rq
import random
import tkinter as tk
from tkinter import messagebox

# Assigning IRL for question database


Then, the key global functions are defined:

1. get_questions(): Fetches questions from the OpenTrivia DB API based on specified parameters, such as difficulty, category, and type, and handles errors if the request or data is invalid. Most of these are set to standard values, but the functions is intendently designed to be flexible, so the code can be adjusted to contain more functionality in the future. Returns a Question instance created from the API response.
2. insertion_sort(): Implements insertion sort to sort players on the leaderboard by their scores in descending order. This function is designed for the final leaderboard display. (see report for detailed rationale on choice of sorting algorithm)

In [ ]:
URL = "https://opentdb.com/api.php"

def get_questions(difficulty, amount=1, category=None, type="multiple"):
    try:
        # Define API request parameters
        params = {
            "difficulty": difficulty,
            "amount": amount,
            "category": category,
            "type": type
        }
        # Send GET request to the API
        response = rq.get(URL, params=params)
        response.raise_for_status()  # Raise an HTTPError if the response status is 4xx or 5xx
        question_data = response.json()  # Parse the JSON response

        # Check if the API returned valid questions
        if question_data.get('response_code') != 0:
            raise ValueError("No questions available for the given parameters.")

        # Create and return a Question instance from the response
        return Question.from_api_response(question_data)

    # Handle network-related errors
    except rq.exceptions.RequestException as e:
        print(f"Error fetching questions: {e}")
        return None

    # Handle invalid data or parameters
    except ValueError as e:
        print(f"Data error: {e}")
        return None

def insertion_sort(players):
    """Sorts the final leaderboard by score (descending) using insertion sort."""
    for i in range(1, len(players)): # Iterate through the players list starting from the second element
        key = players[i]  # Store the current player
        j = i - 1

        # Move players with lower scores one position to the right
        while j >= 0 and players[j].score < key.score:
            players[j + 1] = players[j]
            j -= 1

        # Place the current player in the correct position
        players[j + 1] = key

    # Return the sorted list
    return players



Then, the classes used in the trivia game are defined:

1. Question: Represents a trivia question, including its category, text, answer options, and correct answer. A from_api_response static method is provided to parse API responses and initialize instances. The purpose of having this is a class instead of just working with the output from the "get_questions" function is that it is easier (and more readable) to work with a class and attributes than parsed JSON
2. Player: A base class for players in the game, tracking their name and score. Players can accumulate points using the add_score method.
3. ChildPlayer and AdultPlayer: Subclasses of Player, each with a default difficulty level (easy for children, medium for adults). The hard difficulty is left out since the developpers found even the "medium" difficulty questions to be quite hard to answer
4. TriviaGame: Manages the state of the game, including the list of players and the current round. It allows adding players with their corresponding types (child or adult).

In [10]:
# Classes
class Question:
    """Represents a trivia question, including category, text, options, and the correct answer."""
    def __init__(self, category, question_text, options, correct_answer):
        self.category = category  # Category of the question (e.g., Science, History)
        self.question_text = question_text  # The question text
        self.options = options  # List of possible answer options
        self.correct_answer = correct_answer  # The correct answer for the question

    @staticmethod
    def from_api_response(api_response):
        """Parses a question JSON and initializes a Question instance."""
        question_data = api_response['results'][0] # Extract the first question from the API response (which by default only contains one question)
        
        # Combine incorrect answers with the correct one, then shuffle their order (otherwise the correct answer would always be in the same position, ruining the fun)
        options = question_data['incorrect_answers'] + [question_data['correct_answer']]
        random.shuffle(options)

        # Return a new Question instance with the extracted data
        return Question(
            category=question_data['category'],
            question_text=question_data['question'],
            options=options,
            correct_answer=question_data['correct_answer']
        )

class Player:
    """Represents a player in the game, tracking their name and score."""
    def __init__(self, name):
        self.name = name  # Name of the player
        self.score = 0  # Player's score, initialized to 0

    def add_score(self, points):
        """Adds points to the player's score. Would usually be one, but flexibility is kept for the sake of flexibility in future coding."""
        self.score += points

class ChildPlayer(Player):
    """A specialized Player class for children with an easy default difficulty."""
    DEFAULT_DIFFICULTY = "easy"  # Default difficulty for child players

class AdultPlayer(Player):
    """A specialized Player class for adults with a medium default difficulty."""
    DEFAULT_DIFFICULTY = "medium"  # Default difficulty for adult players

class TriviaGame:
    """Manages the state and players of the trivia game."""
    def __init__(self):
        self.players = []  # List of Player objects participating in the game
        self.round = 1  # Current round of the game, initialized to 1

    def add_player(self, name, player_type):
        """Adds a player to the game, either as a child or an adult."""
        if player_type == "child":
            self.players.append(ChildPlayer(name))  # Add a ChildPlayer instance
        elif player_type == "adult":
            self.players.append(AdultPlayer(name))  # Add an AdultPlayer instance



Then, the TriviaGameGUI class is defined, which creates and manages the graphical user interface (GUI) for the trivia game. The interface includes:

1. A leaderboard displaying player scores.
2. A dynamic question and answer section with options presented as radio buttons.
3. Buttons to submit answers and end the game. It dynamically updates the leaderboard and facilitates progression through rounds, while also handling game termination with a final sorted leaderboard.

In [11]:
class TriviaGameGUI:
    """Handles the graphical user interface for the trivia game."""
    def __init__(self, game):
        self.game = game  # The game logic and state (TriviaGame instance)
        self.current_player_index = 0  # Index of the current player
        self.round_player_index = 0  # Tracks player order within each round
        self.root = tk.Tk()  # Create the main application window
        self.root.title("Trivia Game")  # Set the title of the window

        # Bring the window to the front
        self.root.attributes('-topmost', True)

        # Set the window to full screen
        self.root.state('zoomed')

        # Left Frame for Leaderboard
        self.leaderboard_frame = tk.Frame(self.root, padx=10, pady=10)
        self.leaderboard_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.round_label = tk.Label(self.leaderboard_frame, text=f"Round: {self.game.round}", font=("Arial", 14, "bold"))
        self.round_label.pack()  # Display the current round
        self.leaderboard_label = tk.Label(self.leaderboard_frame, text="Leaderboard", font=("Arial", 14, "bold"))
        self.leaderboard_label.pack()  # Title for the leaderboard
        self.leaderboard_text = tk.Text(self.leaderboard_frame, height=20, width=30, state=tk.DISABLED)
        self.leaderboard_text.pack()  # Area for displaying player scores

        # End Game Button
        self.end_game_button = tk.Button(self.leaderboard_frame, text="End Game", command=self.end_game)
        self.end_game_button.pack(pady=10)  # Button to end the game and display the final leaderboard

        # Main Frame for Question and Options
        self.main_frame = tk.Frame(self.root, padx=20, pady=20)
        self.main_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

        self.question_label = tk.Label(self.main_frame, text="Question will appear here", wraplength=400, font=("Arial", 12))
        self.question_label.pack(pady=10)  # Label to display the question text

        self.options_frame = tk.Frame(self.main_frame)
        self.options_frame.pack(pady=10)  # Frame to hold the answer options
        self.option_vars = []  # Stores the user's selected option
        self.option_buttons = []  # Holds the radio button widgets for options

        # Submit Button
        self.submit_button = tk.Button(self.main_frame, text="Submit Answer", command=self.submit_answer)
        self.submit_button.pack(pady=10)  # Button to submit the selected answer

        self.update_leaderboard()  # Initialize the leaderboard
        self.ask_question()  # Display the first question

        self.root.mainloop()  # Start the GUI event loop

    def update_leaderboard(self):
        """Updates the leaderboard display with current scores."""
        self.round_label.config(text=f"Round: {self.game.round}")  # Update the round label
        self.leaderboard_text.config(state=tk.NORMAL)  # Enable editing for updates
        self.leaderboard_text.delete(1.0, tk.END)  # Clear existing leaderboard content
        for player in self.game.players:  # Loop through players to display their scores
            self.leaderboard_text.insert(tk.END, f"{player.name}: {player.score} points\n")
        self.leaderboard_text.config(state=tk.DISABLED)  # Disable editing after updates

    def ask_question(self):
        """Fetches and displays a question for the current player."""
        player = self.game.players[self.round_player_index]  # Get the current player
        difficulty = player.DEFAULT_DIFFICULTY  # Use the player's default difficulty level
        question = get_questions(difficulty)  # Fetch a question using the API

        if question:
            self.current_question = question  # Store the fetched question
            # Display the question and its category
            self.question_label.config(text=f"{player.name}'s turn\nCategory: {question.category}\n{question.question_text}")
            for btn in self.option_buttons:
                btn.destroy()  # Remove old answer options
            self.option_buttons = []  # Clear old buttons
            self.option_vars = tk.IntVar()  # Variable to track the selected option
            # Create radio buttons for each answer option
            for idx, option in enumerate(question.options, start=1):
                btn = tk.Radiobutton(self.options_frame, text=option, variable=self.option_vars, value=idx)
                btn.pack(anchor="w")
                self.option_buttons.append(btn)  # Add the button to the list
        else:
            # Show an error message if the question cannot be fetched
            messagebox.showinfo("Trivia Game", "There was an error fetching the question using the API. Please check your internet connection, or check the jupyter notebook output for a detailed error message")
            self.next_player()  # Move to the next player

    def submit_answer(self):
        """Handles answer submission and updates player scores."""
        player = self.game.players[self.round_player_index]  # Get the current player
        selected_option = self.option_vars.get()  # Get the selected option

        if selected_option == 0:
            # Show a warning if no option is selected
            messagebox.showwarning("Error", "Please select an option.")
            return

        # Check if the selected option is correct
        if self.current_question.options[selected_option - 1] == self.current_question.correct_answer:
            messagebox.showinfo("Correct!", "You earned 1 point!")  # Notify the player
            player.add_score(1)  # Add 1 point to the player's score
        else:
            # Notify the player of the correct answer
            correct_idx = self.current_question.options.index(self.current_question.correct_answer)
            messagebox.showinfo("Wrong!", f"The correct answer was: {self.current_question.options[correct_idx]}.")

        self.next_player()  # Move to the next player
        self.update_leaderboard()  # Update the leaderboard

    def next_player(self):
        """Advances to the next player's turn or starts a new round if all players have answered."""
        self.round_player_index = (self.round_player_index + 1) % len(self.game.players)
        if self.round_player_index == 0:  # If all players have answered
            self.game.round += 1  # Start a new round
        self.ask_question()  # Ask the next question

    def end_game(self):
        """Ends the game and displays the final leaderboard sorted by score."""
        sorted_players = insertion_sort(self.game.players)  # Sort players by score
        # Create the final leaderboard string
        final_leaderboard = "\n".join([f"{player.name}: {player.score} points" for player in sorted_players])
        # Display the final leaderboard
        messagebox.showinfo("Final Leaderboard", final_leaderboard)
        self.root.destroy()  # Close the application


lastly, the following block defines the main program logic for running the trivia game. It performs the following steps:

1. Initializes the TriviaGame instance to manage players and game state.
2. Prompts the user to input the number of players and validates that it is a positive integer.
3. Collects player names and their types (child or adult), ensuring valid input for each.
4. Launches the TriviaGameGUI to handle the game's graphical interface.

In [ ]:
if __name__ == "__main__":
    # Initialize the game state
    game = TriviaGame()

    # Prompt for the number of players and ensure valid input
    while True:
        try:
            num_players = int(input("Enter the number of players (positive integer): "))
            if num_players > 0:  # Ensure the input is a positive integer
                break
            else:
                print("Invalid input. Please enter a positive integer greater than 0.")
        except ValueError:
            print("Invalid input. Please enter a positive integer.")

    # Collect player details: name and type (child/adult)
    for i in range(num_players):
        name = input(f"Enter the name for Player {i + 1}: ")  # Ask for the player's name
        while True:
            # Prompt for player type and ensure valid input
            player_type = input(f"Is {name} a child or an adult? (child/adult): ").strip().lower()
            if player_type in ["child", "adult"]:  # Accept only 'child' or 'adult'
                break
            else:
                print("Invalid input. Please enter 'child' or 'adult'.")
        
        # Add the player to the game with the specified type
        game.add_player(name, player_type)

    # Launch the graphical user interface
    TriviaGameGUI(game)


Error fetching questions: 429 Client Error: Too Many Requests for url: https://opentdb.com/api.php?difficulty=easy&amount=1&type=multiple
